2. Build a CNN from scratch to classify images in the CIFAR-10 dataset. Focus on designing an effective CNN architecture and using pooling, dropout, and batch normalization.
3. Use an LSTM-based RNN to perform sentiment analysis on movie reviews using the IMDB dataset. Handle sequence padding, embedding, and explore the effect of GRU vs LSTM.
4. Implement a simple Generative Adversarial Network (GAN) to generate synthetic images similar to MNIST digits. Learn how to build Generator and Discriminator, and train them adversarially.                                                               

In [ ]:
# cifar10_cnn.py
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Data: transforms with augmentation for train, normalization for both
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

train_tfms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

train_ds = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tfms)
test_ds  = datasets.CIFAR10(root="./data", train=False, download=True, transform=test_tfms)

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_dl  = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)

# 2) Model: Conv blocks with BN, ReLU; pooling; dropout deeper
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, use_dropout=False, p=0.3):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_ch)
        self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_ch)
        self.dropout = nn.Dropout(p) if use_dropout else nn.Identity()
        self.shortcut = nn.Conv2d(in_ch, out_ch, kernel_size=1, bias=False) if in_ch != out_ch else nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        out = F.relu(out + identity)  # lightweight residual
        return out

class CIFAR10Net(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.layer1 = ConvBlock(3, 64, use_dropout=False)
        self.pool1  = nn.MaxPool2d(2)  # 32->16
        self.layer2 = ConvBlock(64, 128, use_dropout=True, p=0.2)
        self.pool2  = nn.MaxPool2d(2)  # 16->8
        self.layer3 = ConvBlock(128, 256, use_dropout=True, p=0.3)
        self.pool3  = nn.AdaptiveAvgPool2d((1,1))
        self.fc     = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.pool1(x)
        x = self.layer2(x)
        x = self.pool2(x)
        x = self.layer3(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

model = CIFAR10Net().to(device)

# 3) Optimization setup
epochs = 3
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

# 4) Training and evaluation loops
def train_one_epoch(epoch):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_dl:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total
    print(f"Epoch {epoch}: train loss {train_loss:.4f}, acc {train_acc:.4f}")

def evaluate():
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in test_dl:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * labels.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    test_loss = running_loss / total
    test_acc = correct / total
    return test_loss, test_acc

best_acc = 0.0
for epoch in range(1, epochs + 1):
    train_one_epoch(epoch)
    test_loss, test_acc = evaluate()
    scheduler.step()
    print(f"           val loss {test_loss:.4f}, acc {test_acc:.4f}")
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), "cifar10_best.pth")
        print(f"Saved best model with acc {best_acc:.4f}")

print(f"Training complete. Best val acc: {best_acc:.4f}")

Epoch 1: train loss 1.3029, acc 0.5230
           val loss 1.0446, acc 0.6262
Saved best model with acc 0.6262
Epoch 2: train loss 0.8704, acc 0.6931
           val loss 1.2004, acc 0.6030
Epoch 3: train loss 0.6548, acc 0.7724
           val loss 0.6186, acc 0.7839
Saved best model with acc 0.7839
Training complete. Best val acc: 0.7839
